In [5]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
DATA_DIR = 'raw_data'

EXCLUDE DEMOGRPAHICS

In [ ]:
demo_df = None
for i in range(0, 34):
    df = pd.read_parquet(os.path.join(DATA_DIR, f'output_{i}.parquet'))
    demo_i = df[df['omop_table'] == 'person']
    df = df[~(df['omop_table'] == 'person')]
    df.to_parquet(os.path.join(DATA_DIR, f'output_{i}.parquet'))
    if demo_df is None:
        demo_df = demo_i
    else:
        demo_df = pd.concat([demo_df, demo_i], ignore_index=True)

In [ ]:
demo_df = demo_df[['patient_id', 'start', 'code']]

In [ ]:
demo_df['value'] = demo_df['code'].apply(lambda x: x.split('/', 1)[1] if x.startswith(('Gender', 'Race', 'Ethnicity', 'Gender')) else None)
demo_df['code'] = demo_df['code'].apply(lambda x: x.split('/', 1)[0] if x.startswith(('Gender', 'Race', 'Ethnicity', 'Gender')) else x)

In [ ]:
demo_df.to_parquet(os.path.join(DATA_DIR, 'demographics.parquet'))

In [ ]:
# , columns=['patient_id', 'start', 'code', 'value', 'omop_table']

CODES

In [ ]:
codes = []
for i in range(0, 34):
    df = pd.read_parquet(os.path.join(DATA_DIR, f'output_{i}.parquet'))
    codes += list(df['code'].unique())
    codes = list(set(codes))
codes.sort()

In [ ]:
prefixes = [c.split('/', 1)[0] for c in codes]
prefixes = list(set(prefixes))

In [ ]:
len(codes)

BUCKETS

In [ ]:
def try_parse_float(x):
    if isinstance(x, str):
        try:
            return float(x)
        except ValueError:
            if 'pos' in x.lower() or '+' in x.lower(): return 1.0
            elif 'neg' in x.lower(): return 0.0
            else:
                return x
    return x

In [ ]:
def get_df(codes):
    codes_df = None
    for i in range(0, 34):
        df = pd.read_parquet(os.path.join(DATA_DIR, f'output_{i}.parquet'), columns=['patient_id', 'start', 'code', 'value'])
        df = df[df['code'].isin(codes)]
        df['value'] = df['value'].apply(try_parse_float)
        if len(df) == 0:
            continue
        elif codes_df is None: 
            codes_df = df
        else: codes_df = pd.concat([codes_df, df], ignore_index=True)
    codes_df = codes_df[~((codes_df['value'].apply(type)==str) | (codes_df['value'].apply(type) is None))]
    return codes_df

In [ ]:
str_df = None
scale = 10_000 # for 4 decimal places

In [ ]:
codes_dict = {}
for prefix in prefixes:
    print(prefix)
    codes_p = list(filter(lambda x: x.startswith(prefix), codes))
    df = get_df(codes_p)

    for c in codes_p:
        df_c = df[df['code'] == c]

        if df_c['value'].dropna().nunique() <= 1:
            codes_dict[c] = c
        else:
            df_c = df_c.dropna(subset=['value'])
            bucket = np.percentile(df_c['value'], np.arange(0, 110, 10), method='lower')
            bucket = bucket[np.insert(np.diff(bucket) != 0, 0, True)]
            bucket = np.array([
                    np.floor(bucket[0] * scale) / scale,             # floor first
                    *[round(b, ndigits=4) for b in bucket[1:-1]],         # round middle
                    np.ceil(bucket[-1] * scale) / scale              # ceil last
                ])
            codes_dict[c] = bucket[np.concatenate(([True], bucket[1:] != bucket[:-1]))]
    
    with open(f'buckets.pkl', 'wb') as f:
        pickle.dump(codes_dict, f)

In [ ]:
def get_str_df():
    str_df = None
    for i in range(0, 34):
        df = pd.read_parquet(os.path.join(DATA_DIR, f'output_{i}.parquet'), columns=['patient_id', 'start', 'code', 'value'])
        df['value'] = df['value'].apply(try_parse_float)
        
        df = df[(df['value'].apply(type)==str)]

        if len(df) == 0:  continue
        elif str_df is None:  str_df = df
        else: str_df = pd.concat([str_df, df], ignore_index=True)
    
    return str_df

In [ ]:
def convert_value(x):
    if x == 'non reactive' or x == 'non-reactive' or x == 'nonreactive':
        return 'non-reactive'
    elif x == 'not detected' or x == 'none detected':
        return 'not detected'
    elif x == 'occ.':
        return 'occasional'
    elif x == 'dnr':
        return 'normal'
    elif x == 'intermed.':
        return 'intermediate'
    else:
        return x

In [ ]:
IGNORE_VALS = ['NONE SEEN', 'See Note', 'See Below', 'See note', 'See scanned results', 'Not done', 'Comment:', 'Comment',
                'SEE COMMENT', '(NOTE)', 'Detailed information on file in HIS.', 'SEE BELOW','SEE TEXT', 'SEE NOTE', 'SEE COMMENTS',
                'Unable to interpret',  'See Comment', 'Note', 'note', 'tnp', 'not given', 'scanned:  shc/epic media manager', 
                'not applicable', 'indeterminate', 'random', 'equivocal', '-', 'final report']
IGNORE_VALS = [v.lower() for v in IGNORE_VALS]

In [ ]:
str_df = get_str_df()

In [ ]:
str_df['value'] = str_df['value'].apply(lambda x: x.lower() if not x is None else x)
str_df = str_df[~(str_df['value'].isin(IGNORE_VALS))]
str_df['value'] = str_df['value'].apply(convert_value)

In [ ]:
done_dict = {}
for code in str_df['code'].unique():
    df_c = str_df[str_df['code']==code]
    if len(df_c['value'].unique()) <= 1:
        done_dict[code] = code
    else:
        done_dict[code] = df_c['value'].unique()

In [ ]:
with open(f'str_buckets.pkl', 'wb') as f:
    pickle.dump(done_dict, f)

In [ ]:
for i in range(0, 22):
    with open(f'../{i}_buckets.pkl', 'rb') as file:
        buckets_i = pickle.load(file)
        for k in buckets_i.keys():
            if k in codes_dict.keys():
                print(k)
            else:
                codes_dict[k] = buckets_i[k]

In [ ]:
with open(f'buckets.pkl', 'wb') as f:
    pickle.dump(codes_dict, f)

UPDATE DATA

In [1]:
IGNORE_VALS = ['NONE SEEN', 'See Note', 'See Below', 'See note', 'See scanned results', 'Not done', 'Comment:', 'Comment',
                'SEE COMMENT', '(NOTE)', 'Detailed information on file in HIS.', 'SEE BELOW','SEE TEXT', 'SEE NOTE', 'SEE COMMENTS',
                'Unable to interpret',  'See Comment', 'Note', 'note', 'tnp', 'not given', 'scanned:  shc/epic media manager', 
                'not applicable', 'indeterminate', 'random', 'equivocal', '-', 'final report']
IGNORE_VALS = [v.lower() for v in IGNORE_VALS]

In [2]:
def convert_value(x):
    x = x.lower()
    if x == 'non reactive' or x == 'non-reactive' or x == 'nonreactive':
        return 'non-reactive'
    elif x == 'not detected' or x == 'none detected':
        return 'not detected'
    elif x == 'occ.':
        return 'occasional'
    elif x == 'dnr':
        return 'normal'
    elif x == 'intermed.':
        return 'intermediate'
    else:
        return x

In [3]:
def try_parse_float(x):
    if isinstance(x, str):
        try:
            return float(x)
        except ValueError:
            if 'pos' in x.lower() or '+' in x.lower(): return 1.0
            elif 'neg' in x.lower(): return 0.0
            else:
                return x
    return x

In [7]:
def bucket_eventval(event, val, d):
    buckets = d[event]
    if type(buckets) == str: 
        return buckets
    ind = np.searchsorted(buckets, val, side='right')
    if ind == len(buckets):
        eventval = f"{event}|{buckets[ind-1]}-"
    else:
        eventval = f"{event}|{buckets[ind-1]}-{buckets[ind]}"
    return eventval

In [8]:
def bucket_ind(event, val, d):
    buckets = d[event]
    if type(buckets) == str: 
        return 0
    ind = np.searchsorted(buckets, val, side='right')
    return ind

In [9]:
def str_bucket_ind(event, val, d):
    bucket = d[event]
    if type(bucket) == str: 
        return 0
    ind = np.where(bucket == val)[0][0]
    return ind

In [10]:
with open('buckets.pkl', 'rb') as file:
    buckets = pickle.load(file)

In [11]:
with open('str_buckets.pkl', 'rb') as file:
    str_buckets = pickle.load(file)

In [ ]:
for i in range(1, 34):
    df = pd.read_parquet(os.path.join(DATA_DIR, f'output_{i}.parquet'), columns=['patient_id', 'start', 'code', 'value'])
    df['value'] = df['value'].apply(try_parse_float)

    float_df = df[df['value'].apply(type) == float]
    str_df = df[(df['value'].apply(type) == str)]
    none_df = df[df['value'].isna()]

    float_df['eventval'] = float_df.apply(lambda x: bucket_eventval(x['code'], x['value'], buckets), axis=1)
    float_df['bucket'] = float_df.apply(lambda x: bucket_ind(x['code'], x['value'], buckets), axis=1)

    none_code = none_df[none_df['code'].apply(lambda x: type(buckets[x]) == str)]
    none_df = none_df[~(none_df['code'].isin(none_code['code']))]
    none_df = none_df[none_df['code'].isin(str_buckets.keys())]
    none_str = none_df[none_df['code'].apply(lambda x: type(str_buckets[x]) == str)]

    none_df = pd.concat([none_code, none_str], ignore_index=True)
    none_df['eventval'] = none_df['code']
    none_df['bucket'] = 0

    str_df = str_df[~(str_df['value'].str.lower().isin(IGNORE_VALS))]
    str_df['value'] = str_df['value'].apply(convert_value)
    str_df['eventval'] = (str_df['code'] + '|' + str_df['value'])
    str_df['bucket'] = str_df.apply(lambda x: str_bucket_ind(x['code'], x['value'], str_buckets), axis=1)

    df = pd.concat([float_df, str_df, none_df], ignore_index=True)
    
    df['value'] = df['value'].astype(str)
    df.to_parquet(f'../data/output_{i}.parquet')

FOR DATASETS

In [ ]:
AGES = [0, 20, 30, 40, 50, 60, 70, 80, 90]
AGE_LABELS = [f'Age/{AGES[i]}-{AGES[i+1]}' for i in range(0, len(AGES)-1)]
def age_str(x):
    ind = np.searchsorted(AGES, x, 'right')
    return AGE_LABELS[ind-1]

In [ ]:
TASK = 'hypoglycemia'

In [ ]:
TASK = 'hyperkalemia'

In [ ]:
demo_df = pd.read_parquet(f'data/lab_{TASK}/{TASK}_demo.parquet')

In [ ]:
demo_df['race_str'] = 'Race/' + demo_df['race']
demo_df['gender_str'] = 'Gender/' + demo_df['gender']
demo_df['ethnicity_str'] = 'Ethnicity/' + demo_df['ethnicity']
demo_df['age_str'] = demo_df['age'].apply(age_str)

In [ ]:
demo_df.to_parquet(f'data/lab_{TASK}/{TASK}_demo.parquet')

In [ ]:
label_df = pd.read_parquet(f'data/lab_{TASK}/{TASK}_label.parquet')

In [ ]:
label_df['Time'] = label_df.apply(lambda x: x['prediction_time'] if x['value'] else x['Sample_time'], axis=1)

In [ ]:
label_df.to_parquet(f'data/lab_{TASK}/{TASK}_label.parquet')

In [ ]:
df = pd.read_parquet(f'data/lab_{TASK}/{TASK}2.parquet', columns=['patient_id', 'start', 'code', 'value'])
df['value'] = df['value'].apply(try_parse_float)

In [ ]:
float_df = df[df['value'].apply(type) == float]
str_df = df[(df['value'].apply(type) == str)]
none_df = df[df['value'].isna()]

In [ ]:
float_df['eventval'] = float_df.apply(lambda x: bucket_eventval(x['code'], x['value'], buckets), axis=1)
float_df['bucket'] = float_df.apply(lambda x: bucket_ind(x['code'], x['value'], buckets), axis=1)

In [ ]:
none_code = none_df[none_df['code'].apply(lambda x: type(buckets[x]) == str)]
none_df = none_df[~(none_df['code'].isin(none_code['code']))]
none_df = none_df[none_df['code'].isin(str_buckets.keys())]
none_str = none_df[none_df['code'].apply(lambda x: type(str_buckets[x]) == str)]

none_df = pd.concat([none_code, none_str], ignore_index=True)
none_df['eventval'] = none_df['code']
none_df['bucket'] = 0

In [ ]:
str_df = str_df[~(str_df['value'].str.lower().isin(IGNORE_VALS))]
str_df['value'] = str_df['value'].apply(convert_value)
str_df['eventval'] = (str_df['code'] + '|' + str_df['value'])
str_df['bucket'] = str_df.apply(lambda x: str_bucket_ind(x['code'], x['value'], str_buckets), axis=1)

In [ ]:
df = pd.concat([float_df, str_df, none_df], ignore_index=True)
df['value'] = df['value'].astype(str)

In [ ]:
df.to_parquet(f'data/lab_{TASK}/{TASK}2.parquet')

DATA COUNT

In [ ]:
N = 5

In [ ]:
counts = df.groupby('patient_id')['code'].count().reset_index().sort_values('Count')
counts = counts[counts['Count'] > N]

In [ ]:
counts['values'] = (counts['Count']/10).round().astype(int)

In [ ]:
(counts['values'] < 40).sum() / len(counts)

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(counts['values'], bins=range(counts['values'].min(), counts['values'].max() + 2), align='left')
plt.xlabel('Events')
plt.ylabel('Frequency (Total: 116474)')
plt.grid(True, linestyle='--')
plt.show()